## Data preprocessing

In [ ]:
# Chargement des librairies nécessaires
library(ggplot2)
library(tidyverse)
library(gridExtra)
library(GGally)
library(plotly)
library(corrplot)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)

In [ ]:
# Lecture des données
path <- "data/"
song <- read.csv(paste0(path, "spotify_songs.csv"), header = TRUE, sep = ",")

# Premières lignes du jeu de données
head(song)

# Vérification du contenu
summary(song)

In [ ]:
# Check the data types
str(song)

In [ ]:
# Drop the track_id, track_album_id, playlist_id columns
song <- song[, -c(1, 5, 9)]

# As factor the categorical variables track_artist, playlist_genre, playlist_subgenre, key, mode, playlist_name, track_album_name
song$playlist_name <- as.factor(song$playlist_name)
song$track_album_name <- as.factor(song$track_album_name)
song$track_artist <- as.factor(song$track_artist)
song$playlist_genre <- as.factor(song$playlist_genre)
song$playlist_subgenre <- as.factor(song$playlist_subgenre)
song$key <- factor(song$key, levels = c(-1, 0:11), labels = c("No key detected", "C", "C♯/D♭", "D", "D♯/E♭", "E", "F", "F♯/G♭", "G", "G♯/A♭", "A", "A♯/B♭", "B"))
song$mode <- factor(song$mode, levels = c(0, 1), labels = c("minor", "major"))

# track_album_release_date to date (if the full date is not available, we will use the first day of the year)
song$track_album_release_date <- as.Date(ifelse(nchar(song$track_album_release_date) != 10, 
                                                paste0(substr(song$track_album_release_date, 1, 4), "-01-01"), 
                                                song$track_album_release_date), 
                                         format = "%Y-%m-%d")

# Convert the duration_ms to seconds and rename the column to duration_s
song$duration_s <- song$duration_ms / 1000
song$duration_ms <- NULL

# Check the modified dataset
summary(song)
head(song)

In [ ]:
# Check for missing values
colSums(is.na(song))

# Drop the missing values
song <- na.omit(song)

# Check the modified dataset
colSums(is.na(song))

str(song)

# Show the number of data
nrow(song)

In [ ]:
# Check for duplicated data
cat(sum(duplicated(song)),'\n')

# Check for duplicated song names with same artist
cat(sum(duplicated(song[, c("track_name", "track_artist")])), '\n')

In [ ]:
# New data set by removing columns playlist_name, playlist_genre, playlist_subgenre
onlysongs <- song[, -c(6, 7, 8)]

# Check for duplicated data
cat(sum(duplicated(onlysongs)), '\n')

# Drop duplicates
onlysongs <- unique(onlysongs)

# Print number of songs with same name and same artist
cat(sum(duplicated(onlysongs[, c("track_name", "track_artist")])), '\n')

# print rows with name "Something real"
onlysongs[onlysongs$track_name == "Something Real", ]

# Remove duplicates with the same name and artist, keeping only one with the highest popularity
onlysongs <- onlysongs %>%
    group_by(track_name, track_artist) %>%
    filter(track_popularity == max(track_popularity)) %>%
    slice(1) %>%  # If there are ties in popularity, keep just one
    ungroup()

# print rows with name "Something real"
onlysongs[onlysongs$track_name == "Something Real", ]

# Number of value in onlysongs
cat(nrow(onlysongs), '\n')

Descriptive Analysis

In [ ]:
summary(onlysongs)

In [ ]:
#Affichage d'histogrammes représentant la distribution de la popularité des chansons, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s, avec la possibilité de modifier la taille des titres et du texte des axes
options(repr.plot.width=15, repr.plot.height=15)
par(mfrow=c(4, 3))
hist(onlysongs$track_popularity, main="Popularité des chansons", xlab="Popularité", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$danceability, main="Danceability", xlab="Danceability", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$energy, main="Energy", xlab="Energy", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$loudness, main="Loudness", xlab="Loudness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$speechiness, main="Speechiness", xlab="Speechiness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$acousticness, main="Acousticness", xlab="Acousticness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$instrumentalness, main="Instrumentalness", xlab="Instrumentalness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$liveness, main="Liveness", xlab="Liveness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$valence, main="Valence", xlab="Valence", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$tempo, main="Tempo", xlab="Tempo", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$duration_s, main="Durée des chansons", xlab="Durée (s)", ylab="Nombre de chansons", col="lightblue", border="black")

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
par(mfrow=c(2, 3))

# Histogramme pour la variable 'mode'
mode_counts <- table(onlysongs$mode)
barplot(
  mode_counts,
  main = "Distribution de la variable 'mode'",
  xlab = "Mode",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 1 # Labels horizontaux
)

# Histogramme pour la variable 'key'
key_counts <- table(onlysongs$key)
barplot(
  key_counts,
  main = "Distribution de la variable 'key'",
  xlab = "Key",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 1 # Labels horizontaux
)
# Conversion de la variable 'track_album_release_date' en format Date
onlysongs$track_album_release <- as.Date(onlysongs$track_album_release_date)

# Extraction de l'année de sortie
onlysongs$release_year <- format(onlysongs$track_album_release, "%Y")

# Comptage des occurrences par année
release_year_counts <- table(onlysongs$release_year)

# Création de l'histogramme
barplot(
  release_year_counts,
  main = "Distribution des années de sortie des albums",
  xlab = "Année de sortie",
  ylab = "Nombre de morceaux",
  col = "lightblue",
  border = "black",
  las = 2 # Rotation des labels pour les rendre lisibles
)

# Histogramme pour la variable 'playlist_genre'
genre_counts <- table(song$playlist_genre)
barplot(
  genre_counts,
  main = "Distribution de la variable 'playlist_genre'",
  xlab = "Genre de la playlist",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 2 # Rotation des labels pour les rendre lisibles
)

# Histogramme pour la variable 'playlist_subgenre'
subgenre_counts <- table(song$playlist_subgenre)
barplot(
  subgenre_counts,
  main = "Distribution de la variable 'playlist_subgenre'",
  xlab = "Sous-genre de la playlist",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 2 # Rotation des labels pour les rendre lisibles
)

In [ ]:
# Création des catégories pour la variable 'speechiness'
onlysongs$speechiness_category <- cut(
  onlysongs$speechiness,
  breaks = c(0, 0.33, 0.66, 1),
  labels = c("Music", "Speech and music", "Speech"),
  include.lowest = TRUE
)

# Comptage des occurrences dans chaque catégorie
speechiness_counts <- table(onlysongs$speechiness_category)

# Création de l'histogramme
barplot(
  speechiness_counts,
  main = "Distribution des catégories de 'speechiness'",
  xlab = "Catégories de speechiness",
  ylab = "Nombre de morceaux",
  col = "lightblue",
  border = "black"
)

In [ ]:
# Création des catégories pour la variable 'speechiness'
onlysongs$speechiness_category <- cut(
  onlysongs$speechiness,
  breaks = c(0, 0.5, 1),
  labels = c("Vocal", "Instrumental"),
  include.lowest = TRUE
)

# Comptage des occurrences dans chaque catégorie
speechiness_counts <- table(onlysongs$speechiness_category)

# Création de l'histogramme
barplot(
  speechiness_counts,
  main = "Distribution des catégories de 'speechiness'",
  xlab = "Catégories de speechiness",
  ylab = "Nombre de morceaux",
  col = "lightblue",
  border = "black"
)

In [ ]:
# Affichage des colonnes de onlysongs
colnames(onlysongs)

# Affichage de la matrice de corrélation des colonnes popularity, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s
correlation_matrix <- cor(onlysongs[, c("track_popularity", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_s")])
corrplot(correlation_matrix, method = "number")

In [ ]:
# Encodage des artistes en variables numériques
onlysongs$artist_numeric <- as.numeric(as.factor(onlysongs$track_artist))

# Calcul de la corrélation entre l'artiste encodé et la popularité
correlation_artist_popularity <- cor(onlysongs$artist_numeric, onlysongs$track_popularity)

# Affichage de la corrélation
print(correlation_artist_popularity)

In [ ]:
# Scatterplot pour visualiser la relation entre l'accousticness et l'energy
ggplot(onlysongs, aes(x = acousticness, y = energy)) +
  geom_point(aes(color = track_popularity), alpha = 0.5) +
  scale_color_gradient(low = "blue", high = "red") +
  labs(title = "Relation entre l'acousticness et l'energy",
       x = "Acousticness",
       y = "Energy",
       color = "Popularité") +
  theme_minimal()

In [ ]:
# Scatterplot pour visualiser la relation entre l'acousticness et la loudness
ggplot(onlysongs, aes(x = acousticness, y = loudness)) +
  geom_point(aes(color = track_popularity), alpha = 0.5) +
  scale_color_gradient(low = "blue", high = "red") +
  labs(title = "Relation entre l'acousticness et la loudness",
       x = "Acousticness",
       y = "Loudness",
       color = "Popularité") +
  theme_minimal()

In [ ]:
#Scatterplot pour visualiser la relation entre la loudness et l'energy
ggplot(onlysongs, aes(x = loudness, y = energy)) +
  geom_point(aes(color = track_popularity), alpha = 0.5) +
  scale_color_gradient(low = "blue", high = "red") +
  labs(title = "Relation entre la loudness et l'energy",
       x = "Loudness",
       y = "Energy",
       color = "Popularité") +
  theme_minimal()

In [ ]:
# Histogrammes de la popularité des chansons par genre, un histogramme par genre, en utilisant les données "song"
options(repr.plot.width=15, repr.plot.height=10)
par(mfrow=c(3, 2))
for (genre in levels(song$playlist_genre)) {
  hist(song$track_popularity[song$playlist_genre == genre], 
       main = paste("Popularité des chansons dans le genre", genre), 
       xlab = "Popularité", 
       ylab = "Nombre de chansons", 
       col = "lightblue", 
       border = "black")
}

In [ ]:
# Histogrammes de la popularité des chansons par sous-genre, un histogramme par sous-genre, en utilisant les données "song"
options(repr.plot.width=15, repr.plot.height=10)
par(mfrow=c(3, 2))
for (subgenre in levels(song$playlist_subgenre)) {
  hist(song$track_popularity[song$playlist_subgenre == subgenre], 
       main = paste("Popularité des chansons dans le sous-genre", subgenre), 
       xlab = "Popularité", 
       ylab = "Nombre de chansons", 
       col = "lightblue", 
       border = "black")
}

In [ ]:
# Extraction de l'année de sortie
onlysongs$release_year <- format(onlysongs$track_album_release_date, "%Y")

# Calcul de la popularité moyenne par année de sortie
average_popularity_by_year <- onlysongs %>%
  group_by(release_year) %>%
  summarise(mean_popularity = mean(track_popularity, na.rm = TRUE))

# Création du graphique
ggplot(average_popularity_by_year, aes(x = as.numeric(release_year), y = mean_popularity)) +
  geom_line(color = "blue") +
  geom_point(color = "red") +
  labs(title = "Popularité moyenne des musiques par année de sortie",
       x = "Année de sortie",
       y = "Popularité moyenne") +
  theme_minimal()

In [ ]:
# Comptage du nombre de musiques par genre et par année
songs_by_genre_year <- song %>%
    group_by(release_year = format(track_album_release_date, "%Y"), playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Création du graphique
    ggplot(songs_by_genre_year, aes(x = as.numeric(release_year), y = count, color = playlist_genre)) +
        geom_line() +
        labs(title = "Nombre de musiques sorties par genre et par année",
             x = "Année de sortie",
             y = "Nombre de morceaux",
             color = "Genre") +
        theme_minimal()

In [ ]:
# Calcul de la loudness moyenne par genre
average_loudness_by_genre <- song %>%
    group_by(playlist_genre) %>%
    summarise(mean_loudness = mean(loudness, na.rm = TRUE))

# Création du graphique
ggplot(average_loudness_by_genre, aes(x = reorder(playlist_genre, -mean_loudness), y = mean_loudness)) +
    geom_bar(stat = "identity", fill = "lightblue", color = "black") +
    labs(title = "Loudness moyenne par genre de musique",
             x = "Genre de musique",
             y = "Loudness moyenne") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
# Calcul des moyennes par genre pour chaque variable
average_values_by_genre <- song %>%
    group_by(playlist_genre) %>%
    summarise(
        danceability = mean(danceability, na.rm = TRUE),
        energy = mean(energy, na.rm = TRUE),
        loudness = mean(loudness, na.rm = TRUE),
        mode = mean(as.numeric(mode), na.rm = TRUE),
        speechiness = mean(speechiness, na.rm = TRUE),
        acousticness = mean(acousticness, na.rm = TRUE),
        instrumentalness = mean(instrumentalness, na.rm = TRUE),
        liveness = mean(liveness, na.rm = TRUE),
        valence = mean(valence, na.rm = TRUE),
        tempo = mean(tempo, na.rm = TRUE),
        duration_ms = mean(duration_s * 1000, na.rm = TRUE)
    )

# Affichage du tableau
print(average_values_by_genre)

In [ ]:
# Diagramme en barres empliées de la variable "Key" en fonction du genre de la musique
ggplot(song, aes(x = playlist_genre, fill = key)) +
  geom_bar(position = "fill") +
  labs(title = "Répartition de la variable 'Key' par genre de musique",
       x = "Genre de musique",
       y = "Proportion",
       fill = "Key") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
#Diagramme en barre empillés de la variable "Mode" en fonction du genre de la musique
ggplot(song, aes(x = playlist_genre, fill = mode)) +
  geom_bar(position = "fill") +
  labs(title = "Répartition de la variable 'Mode' par genre de musique",
       x = "Genre de musique",
       y = "Proportion",
       fill = "Mode") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
# Calculer le nombre de playlists différentes dans lesquelles chaque chanson apparaît
playlist_count <- song %>%
    group_by(track_name, track_artist) %>%
    summarise(n_playlists = n_distinct(playlist_name),
                        track_popularity = max(track_popularity)) %>%
    ungroup()

# Calcul de la corrélation entre le nombre de playlists et la popularité
correlation <- cor(playlist_count$n_playlists, playlist_count$track_popularity)

# Affichage du résultat
cat("Corrélation entre le nombre de playlists et la popularité :", correlation, "\n")

# Visualisation
ggplot(playlist_count, aes(x = n_playlists, y = track_popularity)) +
    geom_point(alpha = 0.5, color = "blue") +
    geom_smooth(method = "lm", se = FALSE, color = "red") +
    labs(title = "Corrélation entre le nombre de playlists et la popularité",
             x = "Nombre de playlists",
             y = "Popularité de la chanson") +
    theme_minimal()

## 2. Réduction de dimension par Analyse en Composantes Principales (ACP)

Dans cette partie, nous allons effectuer une analyse en composantes principales (ACP) sur les données prétraitées. L'ACP est une technique de réduction de dimension qui permet de projeter les données d'origine dans un espace de dimension inférieure. Nous avons gardé 20 variables et nous allons étudier s'il est possible de réduire la dimensionnalité de ces données tout en préservant un maximum d'information.

### 2.1. Format des données

Ici on ne sélectionne que les variables quantitatives, en y ajoutant une variable qualitative (playlist_genre) pour voir si elle a un impact sur la projection des données. On garde au final 11 variables quantitatives et 1 variable qualitative.

On décide de normaliser les données avant de faire l'ACP car l'ACP est sensible à l'échelle des variables. On va donc centrer et réduire les données.

In [ ]:
# PCA analysis using FactoMineR
song_pca <- song[, c(3, 7, 9:10, 12, 14:20)]

# Perform PCA
pca <- PCA(song_pca,scale.unit = TRUE, graph = FALSE,ncp = 7,quali.sup = 2)

# Afficher le pourcentage de variance expliquée par chaque composante principale
fviz_eig(pca, addlabels = TRUE, ylim = c(0, 40))

# Calculer la variance cumulée
explained_variance <- pca$eig[, 2]  # La deuxième colonne contient le pourcentage de variance expliquée
cumulative_variance <- cumsum(explained_variance) 

# Tracer la variance cumulée
plot(cumulative_variance, xlab = "Nombre de composantes principales", ylab = "Variance cumulée", type = "b")
abline(h = 80, col = "red", lty = 2)  # Ligne horizontale à 80% de variance expliquée

**Interprétation :** 

L’analyse de la variance expliquée montre que les 7 premières composantes principales permettent de représenter 80,1 % de la variance totale du jeu de données. Cela signifie que l’essentiel de l’information contenue dans les 11 variables numériques initiales (comme danceability, energy, speechiness, tempo, etc.) peut être résumé avec seulement 7 dimensions, ce qui représente une réduction significative de la complexité du dataset tout en conservant une bonne qualité descriptive.

Dans cette optique de réduction de dimension, il serait pertinent de conserver ces 7 composantes principales pour la suite des analyses (clustering, visualisation, classification), car elles capturent la structure principale des données tout en éliminant le "bruit".

Dans l’analyse factorielle, nous avons choisi d’interpréter les trois premières composantes principales, qui à elles seules expliquent 45 % de la variance totale. Elles offrent un bon compromis entre lisibilité et pertinence pour une visualisation ou une première analyse des relations entre les variables et les genres musicaux (playlist_genre).

In [ ]:
# Corrélation des variables
corrplot(pca$var$cor, is.corr = FALSE, method = "ellipse")

# Tracer les variables sur le plan factoriel dim 1-2
fviz_pca_var(pca, axes=c(1,2),col.var = "contrib", gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), repel = TRUE) +
  labs(title = "Variables sur le plan factoriel") +
  theme_minimal()

# Tracer les variables sur le plan factoriel dim 1-3
  fviz_pca_var(pca, axes = c(1, 3), col.var = "contrib", 
         gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
         repel = TRUE) +
    labs(title = "Variables sur le plan factoriel (Dim 1-3)") +
    theme_minimal()

# Tracer les variables sur le plan factoriel dim 2-3
  fviz_pca_var(pca, axes = c(2, 3), col.var = "contrib", 
         gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
         repel = TRUE) +
    labs(title = "Variables sur le plan factoriel (Dim 2-3)") +
    theme_minimal()

La table des correlations et les trois graphiques ci-dessus représentent les projections des features sur les trois premières composantes principales nous donne des informations sur la structure des données réduites.

**Composante principale 1 :**
    
Les variables energy (-0.91), loudness (-0.80) et acousticness (+0.72) sont linéairement corrélées avec la première composante principale, en soulignant que energy et loudness sont inversément corrélées avec acousticness. Cela indique que la CP1 oppose les morceaux énergiques, forts en volume et peu acoustiques (ex : rock, électro) aux morceaux calmes, acoustiques et peu énergétiques (ex : folk, classique).

**Composante principale 2 :**

Sur le graphique de gauche, on remarque une opposition des variables instrumentalness (+0.45), duration_s (+0.38) contre danceability (-0.68), valence (-0.62), track_popularity (-0.37), speechiness (-0.39).

Cette composante principale oppose deux profils de morceaux :
    D’un côté, les morceaux instrumentaux, longs et peu populaires (forte contribution de instrumentalness et duration_s), souvent associés à des genres comme le classique ou le jazz.
    De l’autre, les chansons courtes, dansantes, joyeuses et populaires (forte contribution de danceability, valence et track_popularity), typiques de la pop ou de la musique de club.
    Enfin, cette opposition suggère que les morceaux avec des paroles marquées (speechiness) et une structure rythmique engageante (danceability) sont plus susceptibles de générer de la popularité.

**Composantes principales 2 et 3 :**

Sur le graphique au centre, on peut extraire plusieurs informations :
    Les morceaux dansants et joyeux (haute valence) s'opposent dans une moindre mesure aux morceaux de faible tempo. De plus, ces morceaux semblent avoir peu de versions live.
    On observe aussi que les morceaux instrumentaux et longs ont généralement une faible popularité, tandis que les morceaux courts et peu instrumentaux sont souvent plus populaires, ce qui est typique de la musique pop, qui est souvent plus accessible et commerciale.

**Composante principale 3 :**

La troisième composante (CP3) révèle un paradoxe : elle regroupe des morceaux à fort potentiel dansant (danceability) et mood positif (valence), mais qui restent peu populaires (track_popularity). Ces morceaux sont souvent instrumentaux (instrumentalness), longs (duration_s) et à tempo faible (tempo), ce qui les éloigne des standards des charts. Cette composante pourrait représenter des créations artistiques équilibrant danse et complexité, mais peinant à atteindre un large public.

**Remarque:** on peut noter que les valeurs ne sont pas exactement les mêmes que sur le notebook Python. L'ACP sous R ne prend pas forcément la même base que sur Python, ce qui explique les valeurs parfois négatives ou positives.

Pour mieux comprendre à quoi correspond les type morceaux extraits par ces composantes principales, nous allons regarder les morceaux (individus) contribuant le plus à chacune des composantes principales.

In [ ]:
# Extraction des coordonnées des individus sur la dimension 1
dim1_coords <- pca$ind$coord[, 1]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices <- order(dim1_coords)[1:5]
max_indices <- order(dim1_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals <- song[c(min_indices, max_indices), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals$Dim1_Value <- dim1_coords[c(min_indices, max_indices)]

# Réorganiser les colonnes pour mettre Category et Dim1_Value au début
col_order <- c("Category", "Dim1_Value", colnames(extreme_individuals)[1:(ncol(extreme_individuals)-2)])
extreme_individuals <- extreme_individuals[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals)
}



Pour mieux cerner les types de morceaux représentés aux extrémités de la première composante principale (PC1), nous avons identifié les individus (chansons) ayant les contributions les plus élevées, positives comme négatives.

    Du côté des contributions positives, on retrouve majoritairement des morceaux rock, hard rock ou pop rock très énergiques et puissants tels que American Idiot (Green Day), Beauty Queen (BLVK SWVN) ou ATTENTION ATTENTION (Shinedown). Ces morceaux sont caractérisés par une énergie élevée, une forte intensité sonore (loudness) et une faible acoustique, ce qui confirme bien la structure mise en évidence par la CP1. Notons aussi This Is How We Do It (Montell Jordan), un morceau R&B énergique, qui se distingue des autres par son genre mais partage les mêmes caractéristiques acoustiques.

    À l’opposé, les morceaux à contribution très négative sur PC1 sont des titres à forte acoustique, peu énergiques et très faibles en loudness. Il s'agit notamment de sons ambiants, relaxants ou naturels, comme Peaceful Forest ou Tropical Rainforest at Dawn, mais aussi de titres R&B ou indie très doux (Small de chloe moriondo). Ces morceaux incarnent l'autre extrémité de la CP1 : des chansons calmes, acoustiques et à faible énergie, souvent issues de sous-genres comme tropical, indie poptimism ou new jack swing.

Cette opposition renforce l’interprétation de la CP1 comme un axe énergie / intensité sonore vs. calme / acoustique, pertinent pour distinguer deux grandes familles de styles musicaux dans le dataset.


In [ ]:
# Extraction des coordonnées des individus sur la dimension 2
dim2_coords <- pca$ind$coord[, 2]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices_dim2 <- order(dim2_coords)[1:5]
max_indices_dim2 <- order(dim2_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals_dim2 <- song[c(min_indices_dim2, max_indices_dim2), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals_dim2$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals_dim2$Dim2_Value <- dim2_coords[c(min_indices_dim2, max_indices_dim2)]

# Réorganiser les colonnes pour mettre Category et Dim2_Value au début
col_order <- c("Category", "Dim2_Value", colnames(extreme_individuals_dim2)[1:(ncol(extreme_individuals_dim2)-2)])
extreme_individuals_dim2 <- extreme_individuals_dim2[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals_dim2)
}



Côté contributions positives, on retrouve des titres principalement rap et latino, tels que Suge de DaBaby ou LAX de B0nds. Ces morceaux sont :

    courts,
    dansants (haute danceability),
    avec une valence élevée (émotion positive),
    mais également avec un certain niveau de speechiness, notamment pour les titres rap.

Ces morceaux partagent donc des caractéristiques propres aux chansons énergétiques, rythmées et populaires, souvent taillées pour le streaming, avec des formats courts, accrocheurs et directs.

À l’opposé, les morceaux ayant une forte contribution négative à PC2 sont très différents : on retrouve des paysages sonores naturels, ambiants ou instrumentaux comme Rain Forest and Tropical Beach Sound, Caribbean Thunderstorm, ou encore Battlement. Ces titres sont :

    longs,
    instrumentaux (forte instrumentalness),
    avec une faible valence et peu de parole,
    et souvent issus de sous-genres comme tropical, album rock, ou ambient.

Cela confirme l’interprétation initiale de la PC2 comme un axe opposant la musique instrumentale, longue et contemplative à une musique populaire, dansante et rythmée.


In [ ]:
# Extraction des coordonnées des individus sur la dimension 3
dim3_coords <- pca$ind$coord[, 3]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices_dim3 <- order(dim3_coords)[1:5]
max_indices_dim3 <- order(dim3_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals_dim3 <- song[c(min_indices_dim3, max_indices_dim3), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals_dim3$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals_dim3$Dim3_Value <- dim3_coords[c(min_indices_dim3, max_indices_dim3)]

# Réorganiser les colonnes pour mettre Category et Dim3_Value au début
col_order <- c("Category", "Dim3_Value", colnames(extreme_individuals_dim3)[1:(ncol(extreme_individuals_dim3)-2)])
extreme_individuals_dim3 <- extreme_individuals_dim3[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals_dim3)
}



La troisième composante principale met en lumière une tension plus subtile entre deux types de morceaux aux caractéristiques inattendues.

Du côté des contributions négatives, on retrouve des titres de pop et R&B calmes et acoustiques comme raindrops (an angel cried) (Ariana Grande) ou You Are The Reason (Calum Scott). Ces chansons ont :

    une acousticness élevée (acapela, guitare acoustique, piano),
    un tempo légèrement plus rapide que la médiane des morceaux,
    une faible énergie, mais un potentiel émotionnel fort (valence variable).

Comme suggéré par la PC3, ces morceaux rencontrent un certain succès, illustrant un profil de chansons émotionnelles, accessibles et bien produites, souvent interprétées par des artistes grand public au style sobre et expressif.

Les contributions positives, quant à elles, sont largement dominées par des morceaux EDM ou latino instrumentaux, comme I Feel Love ou Chase. Ces morceaux sont :

    longs,
    très instrumentaux,
    énergiques mais souvent moins "accessibles" émotionnellement (valence très élevée mais peu de paroles, structure répétitive).

Ils présentent également une popularité extrêmement faible, allant de 0 à 8. Ces productions s’adressent probablement à un public averti ou sont conçues pour des usages spécifiques (DJ sets, ambiances electro), ce qui les éloigne des standards de la musique grand public.

Ces observations confirment que la PC3 oppose des créations acoustiques à forte charge émotionnelle à des morceaux instrumentaux, électroniques, longs, aux dynamiques parfois complexes ou répétitives.

On remarque le signe des contributions est inversé par rapport à l'analyse en composantes principales (ACP) réalisée sous Python.

## 2. Réduction de dimension par Analyse en Correspondances Multiples (MCA)

## Multiple Factorial Analysis (MCA)

**1. Variables catégorielles à utiliser dans la MCA**
Voici les variables qualitatives que l'on pourrait envisager d'utiliser pour la MCA sur le dataset Spotify :  
- `track_artist`  
- `track_album_name` (attention à trop de modalités rares, peut-être garder que les plus fréquentes ou ne pas inclure à cause du grand nombre d’artistes)  
- `playlist_name`  
- `playlist_genre`  
- `playlist_subgenre`  
- `key`  
- `mode`  

---

**2. Questions que la MCA peut aider à explorer**

**a) Quels sont les groupes/cluster de modalités similaires ?**
- Est-ce que certains genres et sous-genres de playlists s’associent fréquemment ?  
- Certains "modes" (majeur/minor) sont-ils plus fréquents dans certains genres ?  
- Y a-t-il des clés (`key`) musicales qui sont typiques de certains genres ou playlists ?  

La MCA permettra de représenter graphiquement (biplot) ces modalités et d’identifier des associations fortes.

**b) Est-ce que certains artistes ou playlists ont un profil qualitatif particulier ?**
- Par exemple, certains artistes seraient-ils associés à un genre et sous-genre spécifiques, ou à un mode particulier ?  
- Y a-t-il des clusters d’artistes / playlists qui partagent des caractéristiques particulières (clé, mode, genre) ?

**3. Comment interpréter la MCA ici**

- **Axes factoriels** : Chaque axe correspond à une dimension qui résume des associations fortes entre modalités. Par exemple, un axe peut opposer les genres "Rock" à "Pop", ou des clés majeures à mineures.
- **Modalités proches dans l’espace** : Modalités proches signifient qu’elles co-apparaissent souvent dans les observations (ex. certains genres + mode majeur).
- **Observation** : Si tu represents les observations (chansons) dans l’espace MCA, celles proches partagent des profils catégoriels similaires.

---

**4. Utilisations concrètes/academic use cases**

- **Profil des genres musicaux** : Comprendre quels modes/clés/sous-genres caractérisent les genres populaires sur Spotify.  
- **Segmentation qualitative des playlists** : Y a-t-il des types de playlists/musiques qui partagent des caractéristiques qualitatives communes ?  
- **Analyse de diversité** : Mesurer dans quelle mesure certains artistes/genres sont hétérogènes ou homogènes quant à leurs caractéristiques catégorielles.  
- **Préparation à une classification** : Par exemple, combiner le résultat de la PCA (variables numériques) avec la MCA (variables qualitatives) dans une analyse factorielle mixte ou pour enrichir un modèle prédictif.

---

**5. Exemple de questions précises à poser**

- Les genres musicaux sont-ils liés à certaines tonalités ou modes ?  
- Les sous-genres présents dans la même playlist sont-ils proches ou éloignés dans l’espace MCA ?  
- Y a-t-il des clés rares ou des modes minoritaires associés à certains genres uniquement ?  
- Peut-on détecter des groupes de playlists ou artistes avec des profils qualitatifs similaires ?  

---

**En conclusion**
La MCA t’aide surtout à **explorer et visualiser les relations entre variables qualitatives** et leurs modalités sur ton dataset Spotify, ce qui complète bien la PCA sur les variables numériques. C’est une étape utile pour comprendre la structure qualitative de tes données avant d’envisager une modélisation supervisée ou une analyse plus approfondie.

---

Si tu veux, je peux aussi te fournir un exemple de code Python pour réaliser une MCA avec `prince` ou en R avec `FactoMineR` sur ton dataset.

In [ ]:
#MCA avec FactoMineR sur les variables playlist_genre et mode
#song_mca conserve uniquement les variables playlist_genre et mode
song_mca <- song[, c('playlist_genre', 'mode')]
# Perform MCA
mca <- MCA(song_mca, graph = FALSE, ncp = 2)

#Tracer les variables sur le plan factoriel
fviz_mca_var(mca, col.var = "contrib", gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
             repel = TRUE) +
  labs(title = "Variables sur le plan factoriel") +
  theme_minimal()


Clustering

In [ ]:
# Sélection des variables numériques pour l'ACP sur onlysongs
onlysongs_pca_data <- onlysongs %>%
    select(track_popularity, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s)

# Réalisation de l'ACP avec centrage et réduction
onlysongs_pca <- PCA(onlysongs_pca_data, scale.unit = TRUE, graph = TRUE)

# Affichage du pourcentage de variance expliquée par chaque composante principale
fviz_eig(onlysongs_pca, addlabels = TRUE, ylim = c(0, 40))

In [ ]:
# Affichage des individus sur les plans factoriels (Dim 1-2, 2-3, 1-3) pour l'ACP onlysongs
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = "cos2", # Coloration selon la qualité de représentation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "Projection des individus sur le plan factoriel (Dim 1-2)") +
  theme_minimal()

fviz_pca_ind(onlysongs_pca, 
             axes = c(2, 3),
             geom = "point", 
             col.ind = "cos2",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "Projection des individus sur le plan factoriel (Dim 2-3)") +
  theme_minimal()

fviz_pca_ind(onlysongs_pca, 
             axes = c(1, 3),
             geom = "point", 
             col.ind = "cos2",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "Projection des individus sur le plan factoriel (Dim 1-3)") +
  theme_minimal()

Entre R et python la dim 3 est inversée

In [ ]:
# Vérifier le nombre de composantes principales disponibles
n_components <- ncol(onlysongs_pca$ind$coord)

# Méthode du coude (elbow method) pour déterminer le nombre optimal de clusters sur les coordonnées PCA de onlysongs
set.seed(42)
k_values <- 2:10
inertia_pca_onlysongs <- numeric(length(k_values))
# Utilisation des 7 premières composantes principales
pca_coords <- onlysongs_pca$ind$coord[, 1:n_components]

for (i in seq_along(k_values)) {
    k <- k_values[i]
    kmeans_result <- kmeans(pca_coords, centers = k, nstart = 25)
    inertia_pca_onlysongs[i] <- kmeans_result$tot.withinss
}

plot(k_values, inertia_pca_onlysongs, type = "b", pch = 19, col = "blue",
     xlab = "Nombre de clusters (k)", ylab = "Inertie intra-cluster",
     main = "Méthode du coude (K-means sur PCA onlysongs)",
     xlim = c(min(k_values) - 1, max(k_values) + 1))  # Élargir légèrement la plage de l'axe des abscisses

In [ ]:
# K-means sur les coordonnées PCA de onlysongs avec k = 5 clusters
set.seed(42)
kmeans_pca_onlysongs <- kmeans(onlysongs_pca$ind$coord[, 1:n_components], centers = 5, nstart = 25)

# Ajout des labels de cluster au dataset onlysongs
onlysongs$kmeans_pca_cluster <- as.factor(kmeans_pca_onlysongs$cluster)

# Visualisation des clusters sur les deux premières composantes principales
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = onlysongs$kmeans_pca_cluster, 
             palette = "jco", 
             addEllipses = TRUE) +
  labs(title = "K-means (k=5) sur ACP onlysongs (projection Dim 1-2)") +
  theme_minimal()

#affichage sur les dimensions principales 2-3
fviz_pca_ind(onlysongs_pca, 
             axes = c(2, 3),
             geom = "point", 
             col.ind = onlysongs$kmeans_pca_cluster, 
             palette = "jco", 
             addEllipses = TRUE) +
    labs(title = "K-means (k=5) sur ACP onlysongs (projection Dim 2-3)") +
    theme_minimal()

#affichage sur les dimensions principales 1-3
fviz_pca_ind(onlysongs_pca, 
             axes = c(1, 3),
             geom = "point", 
             col.ind = onlysongs$kmeans_pca_cluster, 
             palette = "jco", 
             addEllipses = TRUE) +
    labs(title = "K-means (k=5) sur ACP onlysongs (projection Dim 1-3)") +
    theme_minimal()

In [ ]:
# Passage au format long pour ggplot, en utilisant les clusters obtenus précédemment (kmeans_pca_cluster)
onlysongs_long <- melt(
    cbind(onlysongs[, c("kmeans_pca_cluster")], onlysongs_pca_data),
    id.vars = "kmeans_pca_cluster"
)

# Affichage des boxplots pour chaque variable par cluster
ggplot(onlysongs_long, aes(x = kmeans_pca_cluster, y = value, fill = kmeans_pca_cluster)) +
    geom_boxplot(outlier.size = 0.5) +
    facet_wrap(~ variable, scales = "free_y") +
    labs(title = "Boxplots des variables quantitatives par cluster (K-means PCA onlysongs)",
         x = "Cluster", y = "Valeur") +
    theme_minimal() +
    theme(legend.position = "none")

#agrandir beaucoup la taille des graphes
options(repr.plot.width=40, repr.plot.height=20)

In [ ]:
# Fusionner song avec les clusters de onlysongs (sur track_name et track_artist)
song_with_cluster <- song %>%
    left_join(onlysongs[, c("track_name", "track_artist", "kmeans_pca_cluster")], 
              by = c("track_name", "track_artist"))

# Retirer les musiques qui n'ont pas de cluster
song_with_cluster_noNA <- song_with_cluster %>%
    filter(!is.na(kmeans_pca_cluster))

# Afficher un aperçu du résultat
head(song_with_cluster_noNA[, c("track_name", "track_artist", "kmeans_pca_cluster")])

#affichage du nombre de données total
cat("Nombre total de données dans song_with_cluster_noNA :", nrow(song_with_cluster_noNA), "\n")


In [ ]:
# Création d'une table de contingence : nombre de musiques genre et par cluster
heatmap_data <- song_with_cluster_noNA %>%
    group_by(kmeans_pca_cluster, playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix <- reshape2::acast(heatmap_data, playlist_genre ~ kmeans_pca_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long <- as.data.frame(as.table(heatmap_matrix))
colnames(heatmap_long) <- c("playlist_genre", "kmeans_pca_cluster", "count")

ggplot(heatmap_long, aes(x = kmeans_pca_cluster, y = playlist_genre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

In [ ]:
# Création d'une table de contingence : nombre de musiques par sous-genre et par cluster
heatmap_data <- song_with_cluster_noNA %>%
    group_by(kmeans_pca_cluster, playlist_subgenre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix <- reshape2::acast(heatmap_data, playlist_subgenre ~ kmeans_pca_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long <- as.data.frame(as.table(heatmap_matrix))
colnames(heatmap_long) <- c("playlist_subgenre", "kmeans_pca_cluster", "count")

ggplot(heatmap_long, aes(x = kmeans_pca_cluster, y = playlist_subgenre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))


In [ ]:
# Critère BIC pour déterminer le nombre optimal de composantes pour une GMM sur onlysongs_pca_data
library(mclust)

# Appliquer la GMM sur les données PCA (centrées et réduites)
gmm_model <- Mclust(onlysongs_pca_data, G = 1:10)  # G = nombre de clusters testés

# Afficher le BIC pour chaque nombre de clusters
plot(gmm_model, what = "BIC")

# Nombre optimal de clusters selon le BIC
cat("Nombre optimal de clusters selon le BIC :", gmm_model$G, "\n")

In [ ]:
# Appliquer la GMM sur les données PCA (centrées et réduites) avec 9 clusters
#Applique le modèle VEI (variance égale, covariance identique) sur les données PCA
gmm_9 <- Mclust(onlysongs_pca_data, G = 9, modelNames = "VEI")  # G = nombre de clusters

# Ajouter les labels de cluster GMM au dataset onlysongs
onlysongs$gmm_cluster <- as.factor(gmm_9$classification)

# Afficher un résumé du modèle GMM
summary(gmm_9)

# Afficher la répartition des chansons par cluster GMM
table(onlysongs$gmm_cluster)

# Affichage des clusters GMM sur les dimensions principales 1-2 sans ellipses, juste des points colorés
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = onlysongs$gmm_cluster, 
             palette = "jco", 
             addEllipses = TRUE, # Pas d'ellipses
             pointshape = 19, 
             pointsize = 2, 
             alpha.ind = 0.7) +
  labs(title = "GMM (k=9) sur ACP onlysongs (projection Dim 1-2) - sans ellipses") +
  theme_minimal()

In [ ]:
# Fusionner les clusters GMM avec les données song (sur track_name et track_artist)
song_with_gmm_cluster <- song %>%
    left_join(onlysongs[, c("track_name", "track_artist", "gmm_cluster")], 
              by = c("track_name", "track_artist"))

# Afficher un aperçu du résultat
head(song_with_gmm_cluster[, c("track_name", "track_artist", "gmm_cluster")])

In [ ]:
#création d'une table de contingence : nombre de musiques par genre et par cluster
heatmap_data_gmm <- song_with_gmm_cluster %>%
    group_by(gmm_cluster, playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_gmm <- reshape2::acast(heatmap_data_gmm, playlist_genre ~ gmm_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long_gmm <- as.data.frame(as.table(heatmap_matrix_gmm))
colnames(heatmap_long_gmm) <- c("playlist_genre", "gmm_cluster", "count")
ggplot(heatmap_long_gmm, aes(x = gmm_cluster, y = playlist_genre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par genre et par cluster GMM",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

In [ ]:
# Création d'une table de contingence : nombre de musiques par sous-genre et par cluster GMM
heatmap_data_gmm <- song_with_gmm_cluster %>%
    filter(!is.na(gmm_cluster)) %>%
    group_by(gmm_cluster, playlist_subgenre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_gmm <- reshape2::acast(heatmap_data_gmm, playlist_subgenre ~ gmm_cluster, value.var = "count", fill = 0)

# Format long pour ggplot2
heatmap_long_gmm <- as.data.frame(as.table(heatmap_matrix_gmm))
colnames(heatmap_long_gmm) <- c("playlist_subgenre", "gmm_cluster", "count")

ggplot(heatmap_long_gmm, aes(x = gmm_cluster, y = playlist_subgenre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster GMM",
         x = "Cluster GMM",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

In [ ]:
# Calcul de la matrice de distance euclidienne sur onlysongs_pca_data
dist_matrix <- dist(onlysongs_pca_data, method = "euclidean")

# Réalisation de la classification hiérarchique avec la méthode de Ward
hc_ward <- hclust(dist_matrix, method = "ward.D2")

# Affichage du dendrogramme
plot(hc_ward, labels = FALSE, hang = -1, main = "Dendrogramme (méthode de Ward)", xlab = "", ylab = "Hauteur")

In [ ]:
# Choix du nombre de clusters à découper dans le dendrogramme (par exemple, 3)
n_clusters <- 3
ward_clusters <- cutree(hc_ward, k = n_clusters)

# Ajout des labels de cluster au dataset onlysongs
onlysongs$ward_cluster <- as.factor(ward_clusters)

# Visualisation des clusters Ward sur les deux premières composantes principales de l'ACP
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = onlysongs$ward_cluster, 
             palette = "jco", 
             addEllipses = TRUE) +
  labs(title = "Classification hiérarchique de Ward (k=5) sur ACP onlysongs (projection Dim 1-2)") +
  theme_minimal()

In [ ]:
# Fusionner les clusters Ward avec les données song (sur track_name et track_artist)
song_with_ward_cluster <- song %>%
    left_join(onlysongs[, c("track_name", "track_artist", "ward_cluster")], 
              by = c("track_name", "track_artist"))

# Afficher un aperçu du résultat
head(song_with_ward_cluster[, c("track_name", "track_artist", "ward_cluster")])

In [ ]:
#création d'une table de contingence : nombre de musiques par genre et par cluster
heatmap_data_ward <- song_with_ward_cluster %>%
    group_by(ward_cluster, playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_ward <- reshape2::acast(heatmap_data_ward, playlist_genre ~ ward_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long_ward <- as.data.frame(as.table(heatmap_matrix_ward))
colnames(heatmap_long_ward) <- c("playlist_genre", "ward_cluster", "count")

ggplot(heatmap_long_ward, aes(x = ward_cluster, y = playlist_genre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par genre et par cluster Ward",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

In [ ]:
# Création d'une table de contingence : nombre de musiques par sous-genre et par cluster Ward
heatmap_data_ward <- song_with_ward_cluster %>%
    group_by(ward_cluster, playlist_subgenre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_ward <- reshape2::acast(heatmap_data_ward, playlist_subgenre ~ ward_cluster, value.var = "count", fill = 0)

# Format long pour ggplot2
heatmap_long_ward <- as.data.frame(as.table(heatmap_matrix_ward))
colnames(heatmap_long_ward) <- c("playlist_subgenre", "ward_cluster", "count")

ggplot(heatmap_long_ward, aes(x = ward_cluster, y = playlist_subgenre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster Ward",
         x = "Cluster Ward",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))